In [1]:
import datetime

### Classe Cliente

Do jeito que fiz, a classe cliente pode interagir com diferentes lojas. Podemos pensar nela como uma espécie de perfil simplificado em um *marketplace* de aluguel de bicicletas. Assim, a função de calcular o total depois de um aluguel de bicicletas é feita por um método dessa mesma classe.

Quanto à promoção de aluguel em família, decidi que a promoção só é válida para alugueis do mesmo tipo de 3 bicicletas ou mais. No entanto, é possível devolver separadamente as bicicletas alugadas na promoção, sendo o desconto aplicado separadamente a cada uma delas. Assim, posso devolver as bicicletas com valor promocional em dias diferentes, e a restrição de que o tipo de aluguel seja o mesmo impede fraudes (por exemplo, conseguir desconto de 20% no aluguel de 1 semana alugando 2 por hor

In [2]:
class Cliente:
    
    def __init__(self, nome, cpf):
        self.__nome = nome
        self.__cpf = cpf
        self.__alugadas = {}
    
    # getter de cpf, pois este é um atributo privado
    def getCPF(self):
        return self.__cpf
    
    # getter de todas as bicicletas atualmente alugadas pelo cliente, com a respectiva loja de origem
    def getAlugadas(self):
        return self.__alugadas
    
    def ver_estoque(self, loja):
        disp = loja.getDisponibilidade()
        return [bicicleta for bicicleta in disp if disp[bicicleta]]
    
    def ver_preco(self, loja):
        return loja.getPreco()
   
    # alugar emprega o método de aluguel da loja em que se pretende realizar a transação
    def alugar(self, loja, bicicletas, data, modo = 'h'):
        alugadas_temp = []
        cnpj_loja = loja.getCNPJ()
        # bicicletas é uma lista, e alugamos uma de cada vez com o método da loja
        for bicicleta in bicicletas:
            alugou = loja.alugar(self, bicicleta, data, modo)
            #a bicicleta só entra em self.__alugadas se, de fato, for alugada
            if alugou:
                alugadas_temp.append(bicicleta)
            else:
                print(f'Cliente {self.__nome}: Não foi possível alugar a bicicleta {bicicleta}!')
        numero_alugadas = len(alugadas_temp)
        promocao = numero_alugadas >= 3
        # self.__alugadas é um dict com chaves da forma (bicicleta, cnpj_loja), cujos valores são se vale ou não a promção
        self.__alugadas.update({(bicicleta, cnpj_loja) : promocao for bicicleta in alugadas_temp})
        print(f'Cliente {self.__nome}: Foram alugadas {numero_alugadas} bicicletas!')
        if promocao:
            print(f'Cliente {self.__nome}: As bicicletas alugadas têm 30% de desconto em seu aluguel!')
            
    
    def devolver(self, loja, bicicletas, data):
        cnpj_loja = loja.getCNPJ()
        total = 0
        # aplica-se a devolução em loja para cada bicicleta
        for bicicleta in bicicletas:
            devolveu = loja.devolver(self, bicicleta, data)
            if devolveu:
                self.__alugadas.pop((bicicleta, cnpj_loja))
                total += devolveu
            else:
                print(f'Cliente {self.__nome}: A bicicleta {bicicleta} não pôde ser devolvida!')
        print(f'Preço total do aluguel na loja {loja.getNome()} = R$ {total:.2f}')
        
    

### Classe Loja

A classe loja tem atributos secretos. Como é possível fazer várias lojas diferentes, deixei os preços por modo de aluguel como sendo atributo que pode variar de objeto por objeto, mas mantive como padrão os preços estipulados pelo projeto. Há um atributo para o catálogo, um atributo para a disponibilidade (todas as bicicletas estão disponível quando uma instância de Loja é criada, pois ainda não houve aluguéis), um atributo que é um registro, por CPF, dos clientes e das bicicletas por eles alugadas (incluindo data e preço do aluguel), e um atributo de CPNJ - que atua como identificador único das lojas.

In [3]:
class Loja:
    
    def __init__(self, nome, catalogo, cnpj, modos_preco = {'h': 5, 'd': 25, 's': 100}):
        self.__nome = nome
        self.__catalogo = catalogo
        self.__disponibilidade = {bicicleta: True for bicicleta in catalogo}
        self.__info = {}
        self.__cnpj = cnpj
        self.__modos_preco = modos_preco
    
    # getters para os atributos privados
    def getNome(self):
        return self.__nome
    
    def getDisponibilidade(self):
        return self.__disponibilidade
    
    def getPreco(self):
        return self.__modos_preco
    
    def getCNPJ(self):
        return self.__cnpj
    
    # antes de alugar, um cliente deve se cadastrar na loja
    def cadastrar_cliente(self, cliente):
        #cada cliente é registrado como um dict, cujas chaves serão as bicicletas alugadas, e os valores, os dados do aluguel
        if cliente.getCPF() not in self.__info:
            self.__info[cliente.getCPF()] = {}
    
    def alugar(self, cliente, bicicleta, data, modo = 'h'):
        # se o cliente não for cadastrado, não é possível fazer o aluguel
        if cliente.getCPF() not in self.__info:
            print(f'Loja {self.__nome}: CPF não cadastrado!')
            return None
        # verifica se a bicicleta está no catálogo e se está disponível
        if bicicleta in self.__catalogo and self.__disponibilidade.get(bicicleta):
            # registra-se a bicicleta no cadastro do cliente, e muda altera a disponibilidade da bicicleta
            self.__info[cliente.getCPF()][bicicleta] = (data, modo)
            self.__disponibilidade[bicicleta] = False
            # o retorno True indica que o aluguel foi efetuado com sucesso
            return True
        # tratando os outros casos possíveis. Note que, se a bicicleta não estiver disponível, o retorno é None - que é falsy.
        elif bicicleta not in self.__catalogo:
            print(f'Loja {self.__nome}: Não foi possível alugar a bicicleta {bicicleta}, pois ela não consta no catálogo!')
        elif not self.__disponibilidade[bicicleta]:
            print(f'Loja {self.__nome}: {bicicleta} não está disponível!')
    
    def devolver(self, cliente, bicicleta, data):
        # antes de tudo, verifica-se se o cliente está nos registros de aluguel. Note, novamente, que None é falsy.
        if cliente.getCPF() not in self.__info:
            print(f'Loja {self.__nome}: CPF não cadastrado!')
            return None
        cnpj_loja = self.__cnpj
        registro_alugadas = self.__info[cliente.getCPF()]
        cliente_alugadas = cliente.getAlugadas()
        # verifica se a bicicleta está nos registros do cliente e da loja
        if (bicicleta, cnpj_loja) in cliente_alugadas and bicicleta in registro_alugadas:
            data_aluguel, modo = registro_alugadas.pop(bicicleta)
            # infelizmente, objetos do tipo timedelta só têm dias, segundos e microssegundos. Assim, conversão de tempo é necessária.
            tempo_aluguel = data - data_aluguel
            dias_aluguel = tempo_aluguel.days
            tempo_minutos = tempo_aluguel.seconds // 60
            # modos_tempo é a variável que recupera a quantidade relevante de tempo para cada modo de aluguel
            modos_tempo = {'h' : tempo_minutos // 60, 'd': dias_aluguel, 's': dias_aluguel // 7}
            # os desvios calculam quando não se completa uma unidade relevante de tempo no aluguel, mas mesmo assim se cobra essa unidade - com alguma tolerância.
            desvio_h = 1 if tempo_minutos % 60 >= 20 or modos_tempo['h'] == 0 else 0
            desvio_d = 1 if modos_tempo['h'] % 24 >= 12 or dias_aluguel == 0 else 0
            desvio_s = 1 if dias_aluguel % 7 >= 2 or modos_tempo['s'] == 0 else 0
            # desvios para cada modo de aluguel
            desvio = {'h': desvio_h, 'd': desvio_d, 's': desvio_s }
            promocao = cliente_alugadas[(bicicleta, cnpj_loja)]
            desconto = 0.3 if promocao else 0
            preco = (modos_tempo[modo] + desvio[modo])*self.__modos_preco[modo]*(1 - desconto)
            # depois de devolvida, a bicicleta novamente fica disponível
            self.__disponibilidade[bicicleta] = True
            # note que se preco > 0, o retorno é truthy
            return preco
        else:
            print(f'Loja {self.__nome}: A bicicleta {bicicleta} não consta na lista de alugadas do cliente, ou há um problema no sistema!') 

Abaixo, alguns teste que fiz para verificar se o programa estava funcionando de acordo com o requerido. Se quiser, pode testar algumas coisas também!

In [4]:
cliente1 = Cliente('Ricardo', '00000000000')

In [5]:
loja1 = Loja('rei dos pedais', ['caloi', 'colli', 'sense', 'nathor'], '111111111')

In [6]:
loja1.cadastrar_cliente(cliente1)

In [11]:
cliente1.alugar(loja1, ['caloi', 'colli', 'sense'], datetime.datetime(2023, 7, 12), 'd')

Loja rei dos pedais: caloi não está disponível!
Cliente Ricardo: Não foi possível alugar a bicicleta caloi!
Loja rei dos pedais: colli não está disponível!
Cliente Ricardo: Não foi possível alugar a bicicleta colli!
Loja rei dos pedais: sense não está disponível!
Cliente Ricardo: Não foi possível alugar a bicicleta sense!
Cliente Ricardo: Foram alugadas 0 bicicletas!


In [10]:
cliente1.alugar(loja1, ['nathor'], datetime.datetime(2023,7,13), 'd')

Loja rei dos pedais: nathor não está disponível!
Cliente Ricardo: Não foi possível alugar a bicicleta nathor!
Cliente Ricardo: Foram alugadas 0 bicicletas!


In [12]:
loja1.getDisponibilidade()

{'caloi': False, 'colli': False, 'sense': False, 'nathor': False}

In [13]:
cliente1.getAlugadas()

{('nathor', '111111111'): False,
 ('caloi', '111111111'): True,
 ('colli', '111111111'): True,
 ('sense', '111111111'): True}

In [14]:
cliente1.devolver(loja1, ['caloi', 'colli', 'sense', 'nathor'], datetime.datetime(2023,7, 15))

Preço total do aluguel na loja rei dos pedais = R$ 207.50
